In [4]:
############################## version ##############################
# python version: 3.6.8
# tensorflow version: 1.15 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import re
import os
import yfinance as yf
import tensorflow as tf 
import ipynb
###from ipynb.fs.full.model import *

In [5]:
############################## Data download & manipulcation #####################
#### settings
pd.set_option('max_columns',None)
current_wd = os.getcwd() 
data_dir = current_wd + '/downloads'

In [6]:
#### input data from csv file
rawdata = pd.read_csv(data_dir+'/Candles.csv')
rawdata.head()

,__time,marketplace,raw_pair,price_open,price_high,price_low,price_close,price_vwap,trade_count,volume,volume_avg,volume_max,volume_min
0,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitfinex,ZCN/ETH,0.000945,0.000945,0.000945,0.000945,0.000945,1,400.0,400.000,400.00,400.00
1,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/BTC,0.000069,0.000069,0.000067,0.000067,0.000068,11570,5795909.0,241496.200,724703.00,55844.00
2,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/ETH,0.001977,0.001977,0.001896,0.001906,0.001922,1963,449248.0,18718.666,53773.00,887.00
3,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/BNB,0.018077,0.018077,0.016831,0.016831,0.017490,275,173328.7,8666.435,37692.70,673.20
4,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/USDT,0.271170,0.274445,0.267342,0.269371,0.270710,3402,2161824.0,90076.000,280033.28,10026.78


In [7]:
#### remove null value case ####
rawdata.isnull().sum()

__time           0
marketplace      0
raw_pair       153
price_open       0
price_high       0
price_low        0
price_close      0
price_vwap       0
trade_count      0
volume           0
volume_avg       0
volume_max       0
volume_min       0
dtype: int64

In [8]:
rawdata=rawdata[~rawdata.raw_pair.isnull()].copy()

In [9]:
####### Create new variable : totamt = price_vwap * volume for each raw pair
rawdata = rawdata.assign(totamt = rawdata.price_vwap * rawdata.volume)

In [10]:
####### Create dtime varialble (short version of datetime - only date)
rawdata = rawdata.assign(dtime = pd.to_datetime(list(map(lambda x:re.search(r'... ... \d\d \d\d\d\d',x).group(),rawdata.__time))))
rawdata.head()

,__time,marketplace,raw_pair,price_open,price_high,price_low,price_close,price_vwap,trade_count,volume,volume_avg,volume_max,volume_min,totamt,dtime
0,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitfinex,ZCN/ETH,0.000945,0.000945,0.000945,0.000945,0.000945,1,400.0,400.000,400.00,400.00,0.378000,2019-03-15
1,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/BTC,0.000069,0.000069,0.000067,0.000067,0.000068,11570,5795909.0,241496.200,724703.00,55844.00,392.270691,2019-03-15
2,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/ETH,0.001977,0.001977,0.001896,0.001906,0.001922,1963,449248.0,18718.666,53773.00,887.00,863.368490,2019-03-15
3,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/BNB,0.018077,0.018077,0.016831,0.016831,0.017490,275,173328.7,8666.435,37692.70,673.20,3031.505963,2019-03-15
4,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ZRX/USDT,0.271170,0.274445,0.267342,0.269371,0.270710,3402,2161824.0,90076.000,280033.28,10026.78,585227.158858,2019-03-15


In [11]:
###### Only consider pairs (marketplace,raw_pair) whose number of obervations are at least min_n_obs for the further analysis
allpairs_ncount = rawdata.groupby(['marketplace','raw_pair'])['totamt'].count()
len(allpairs_ncount)

17450

In [12]:
min_n_obs = 300
eligiblepairs = allpairs_ncount[allpairs_ncount>min_n_obs]
len(eligiblepairs)

1565

In [13]:
####only keep those pairs in the raw data
rawdata['mkt_pair'] = list(zip(rawdata.marketplace, rawdata.raw_pair))
rawdata = rawdata[rawdata.mkt_pair.isin(eligiblepairs.index)]

In [14]:
#### Leave the largest marketplace and payment currency (in denominator) in terms of totamt for each raw_pair i.e., choose the most representative (currency,market,payment currency)
mkt_pair_totamt = rawdata.groupby('mkt_pair')['totamt'].sum()
mkt_pair_index_split = pd.Series(list(map(lambda x:x[1].split(r"/",1),mkt_pair_totamt.index)))
mkt_pair_index_split_numerator = pd.Series(list(map(lambda x:x[0],mkt_pair_index_split)),index=mkt_pair_totamt.index)
mkt_pair_final_list = [mkt_pair_totamt[mkt_pair_index_split_numerator == x].nlargest(1).index[0] for x in set(mkt_pair_index_split_numerator)]
mkt_pair_final_list

[('binance', 'POWR/BNB'),
 ('binance', 'MTL/USDT'),
 ('bittrex', 'SIB/BTC'),
 ('bittrex', 'BORA/BTC'),
 ('binance', 'THETA/USDT'),
 ('binance', 'NXS/BNB'),
 ('bitfinex', 'RCN/USD'),
 ('binance', 'BEAM/USDT'),
 ('binance', 'NAV/BTC'),
 ('binance', 'VIBE/ETH'),
 ('bittrex', 'SPC/BTC'),
 ('bittrex', 'OCEAN/USDT'),
 ('bitfinex', 'UTNP/USD'),
 ('binance', 'BTT/TRX'),
 ('binance', 'COS/USDT'),
 ('binance', 'HIVE/USDT'),
 ('binance', 'DASH/USDT'),
 ('binance', 'NKN/USDT'),
 ('binance', 'REN/BNB'),
 ('bitfinex', 'XCHF/USD'),
 ('bittrex', 'BTU/BTC'),
 ('binance', 'CTSI/USDT'),
 ('bittrex', 'CRW/BTC'),
 ('bitfinex', 'DRGN/USD'),
 ('binance', 'POLY/BNB'),
 ('binance', 'ENU/USD'),
 ('binance', 'BCD/ETH'),
 ('binance', 'SYS/BTC'),
 ('binance', 'BGBP/USDC'),
 ('bitfinex', 'ANT/USD'),
 ('binance', 'PPT/ETH'),
 ('bitfinex', 'CNN/USD'),
 ('bitfinex', 'ORS/USD'),
 ('binance', 'ATOM/USDT'),
 ('binance', 'MITH/USDT'),
 ('bitfinex', 'ELF/USD'),
 ('bittrex', 'OK/BTC'),
 ('bittrex', 'ZEN/USD'),
 ('bittrex', 

In [15]:
len(mkt_pair_final_list)

369

In [16]:
rawdata = rawdata[rawdata.mkt_pair.isin(mkt_pair_final_list)] 

In [17]:
#### identify duplicate entries and drop (more than 1 row for the same day)
investigate_dup = (rawdata.groupby(['mkt_pair','dtime'])['totamt'].count() > 1).groupby('mkt_pair').sum()
drop_list = list(investigate_dup[investigate_dup > 0].index)
for i in drop_list:
    mkt_pair_final_list.remove(i)
rawdata = rawdata[rawdata.mkt_pair.isin(mkt_pair_final_list)]  

In [18]:
#### Create price table for each mkt_pair 
price_table = rawdata.pivot(index='dtime',columns='mkt_pair',values='price_vwap')
price_table.shape

(730, 367)

In [39]:
import import_ipynb
import model_final2
from model_final_nydig import *

importing Jupyter notebook from model_final_nydig.ipynb


In [40]:
##############################Model estimation######################################
######## create an DailyPortfolio instance - yield table which includes TSLA
TSLA_DP = DailyPortfolio(price_table,"TSLA")

In [41]:
TSLA_DP.yield_table

,"(binance, ADA/USDT)","(binance, ADX/ETH)","(binance, AE/BNB)","(binance, AION/USDT)","(binance, ALGO/USDT)","(binance, AMB/BNB)","(binance, ANKR/USDT)","(binance, APPC/BNB)","(binance, ARDR/USDT)","(binance, ARK/ETH)","(binance, ARN/ETH)","(binance, ARPA/USDT)","(binance, AST/ETH)","(binance, ATOM/USDT)","(binance, BAND/USDT)","(binance, BAT/USDT)","(binance, BCD/ETH)","(binance, BCH/USDT)","(binance, BCPT/BNB)","(binance, BEAM/USDT)","(binance, BGBP/USDC)","(binance, BLZ/BNB)","(binance, BNB/IDRT)","(binance, BNT/USDT)","(binance, BRD/BNB)","(binance, BTCDOWN/USDT)","(binance, BTCUP/USDT)","(binance, BTS/USDT)","(binance, BTT/TRX)","(binance, BUSD/NGN)","(binance, CDT/ETH)","(binance, CELR/USDT)","(binance, CHR/USDT)","(binance, CHZ/USDT)","(binance, CMT/BNB)","(binance, COCOS/USDT)","(binance, COS/USDT)","(binance, COTI/USDT)","(binance, CTSI/USDT)","(binance, CTXC/USDT)","(binance, CVC/USDT)","(binance, DASH/USDT)","(binance, DATA/USDT)","(binance, DENT/USDT)","(binance, DGD/ETH)","(binance, DLT/BNB)","(binance, DOCK/USDT)","(binance, DOGE/USDT)","(binance, DREP/USDT)","(binance, DUSK/USDT)","(binance, ENG/ETH)","(binance, ENJ/USDT)","(binance, ENU/USD)","(binance, EOS/USDT)","(binance, ERD/USDT)","(binance, ETC/USDT)","(binance, ETHOS/ETH)","(binance, EUR/USDT)","(binance, EVX/ETH)","(binance, FET/USDT)","(binance, FTM/USDT)","(binance, FTT/USDT)","(binance, FUEL/BTC)","(binance, FUN/USDT)","(binance, GAS/BTC)","(binance, GO/BNB)","(binance, GRS/ETH)","(binance, GTO/USDT)","(binance, GVT/ETH)","(binance, GXC/ETH)","(binance, GXS/USDT)","(binance, HBAR/USDT)","(binance, HC/USDT)","(binance, HIVE/USDT)","(binance, HOT/USDT)","(binance, ICX/USDT)","(binance, INS/ETH)","(binance, IOST/TUSD)","(binance, IOTX/USDT)","(binance, KAVA/USDT)","(binance, KEY/USDT)","(binance, LEND/ETH)","(binance, LINK/USDT)","(binance, LSK/USDT)","(binance, LTC/USDT)","(binance, LTO/USDT)","(binance, LUN/BTC)","(binance, MATIC/USDT)","(binance, MBL/USDT)","(binance, MCO/USDT)","(binance, MDA/ETH)","(binance, MFT/USDT)","(binance, MIOTA/USDT)","(binance, MITH/USDT)","(binance, MTH/ETH)","(binance, MTL/USDT)","(binance, NANO/USDT)","(binance, NAS/BNB)","(binance, NAV/BTC)","(binance, NCASH/ETH)","(binance, NEBL/BNB)","(binance, NEO/USDT)","(binance, NKN/USDT)","(binance, NPXS/USDT)","(binance, NULS/USDT)","(binance, NXS/BNB)","(binance, OAX/ETH)","(binance, OGN/USDT)","(binance, OMG/USDT)","(binance, ONE/USDT)","(binance, ONG/USDT)","(binance, ONT/USDT)","(binance, OST/BNB)","(binance, PAX/USDT)","(binance, PERL/USDT)","(binance, PHB/TUSD)","(binance, PIVX/BNB)","(binance, POA/ETH)","(binance, POE/ETH)","(binance, POLY/BNB)","(binance, POWR/BNB)","(binance, PPT/ETH)","(binance, QKC/ETH)","(binance, QLC/ETH)","(binance, QSP/BNB)","(binance, QTUM/USDT)","(binance, RDN/BNB)","(binance, REN/BNB)","(binance, RLC/USDT)","(binance, RRB/USDT)","(binance, RVN/USDT)","(binance, SKY/BNB)","(binance, SNM/ETH)","(binance, SOL/BUSD)","(binance, STEEM/BNB)","(binance, STORM/BNB)","(binance, STPT/USDT)","(binance, STRAT/ETH)","(binance, STX/USDT)","(binance, SYS/BTC)","(binance, TCT/USDT)","(binance, TFUEL/USDT)","(binance, THETA/USDT)","(binance, TNT/ETH)","(binance, TOMO/USDT)","(binance, TROY/USDT)","(binance, TRX/USDT)","(binance, TUSD/USDT)","(binance, USDC/USDT)","(binance, USDS/USDT)","(binance, USDT/IDRT)","(binance, VET/USDT)","(binance, VIA/BNB)","(binance, VIB/ETH)","(binance, VIBE/ETH)","(binance, VITE/USDT)","(binance, WABI/BNB)","(binance, WAN/USDT)","(binance, WAVES/USDT)","(binance, WIN/USDT)","(binance, WINK/TRX)","(binance, WPR/ETH)","(binance, WRX/USDT)","(binance, WTC/USDT)","(binance, XEM/BNB)","(binance, XLM/USDT)","(binance, XMR/USDT)","(binance, XRP/USDT)","(binance, XTZ/USDT)","(binance, XZC/XRP)","(binance, ZEC/USDT)","(binance, ZIL/USDT)","(binance-us, VTHO/USDT)","(bitfinex, ABYSS/USD)","(bitfinex, AGI/USD)","(bitfinex, AID/USD)","(bitfinex, AMPL/USD)","(bitfinex, ANT/USD)","(bitfinex, AUC/USD)","(bitfinex, AVT/USD)","(bitfinex, BCHABC/U

In [42]:
######create date variable and training/testing set - raw pair which has more than 99% of non-na yield value will be included#######
TSLA_DP_DV = DateVar(TSLA_DP,port_date="2021-03-01",min_obs_ratio = 0.99)

In [43]:
####training set and testing set for the analysis######
TSLA_DP_DV.generate_index([TSLA_DP_DV.ini_train_index,TSLA_DP_DV.ini_test_index])

[(DatetimeIndex(['2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21',
                 '2019-03-22', '2019-03-25', '2019-03-26', '2019-03-27',
                 '2019-03-28', '2019-03-29',
                 ...
                 '2021-02-12', '2021-02-16', '2021-02-17', '2021-02-18',
                 '2021-02-19', '2021-02-22', '2021-02-23', '2021-02-24',
                 '2021-02-25', '2021-02-26'],
                dtype='datetime64[ns]', length=492, freq=None),
  DatetimeIndex(['2021-03-01'], dtype='datetime64[ns]', freq='B')),
 (DatetimeIndex(['2019-03-18', '2019-03-19', '2019-03-20', '2019-03-21',
                 '2019-03-22', '2019-03-25', '2019-03-26', '2019-03-27',
                 '2019-03-28', '2019-03-29',
                 ...
                 '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19',
                 '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25',
                 '2021-02-26', '2021-03-01'],
                dtype='datetime64[ns]', length=493, freq=Non

In [44]:
######## run with a default hyperparameter setting - dep: TSLA yield, ind: raw pairs######
TSLA_DP_DV_MF = ModelFit(TSLA_DP_DV)
TSLA_DP_DV_MF.MSE_est

[0.002025749694440065,
 0.002028054722461208,
 0.0020302683806908005,
 0.0020332979372268294,
 0.0020329961891218406,
 0.0020327323490348377,
 0.002040206841467097,
 0.0021139473001084326,
 0.002109718125408757,
 0.0021095586912771346]

In [46]:
TSLA_DP_DV_MF.y_pred

[0.007756508,
 0.012075126,
 0.011759041,
 -0.004738485,
 0.005762784,
 0.016821828,
 -0.0018937119,
 -0.0027481837,
 0.00093043706,
 0.0070874845]

In [47]:
TSLA_DP_DV_MF.y_true

[0.06355291291751475,
 -0.04452763743232491,
 -0.04842373713092618,
 -0.04862218183112754,
 -0.03779928897736162,
 -0.05844972237400059,
 0.1964121085077153,
 -0.008195046455057975,
 0.04721129560609172,
 -0.00839050217557713]

In [48]:
######## run with optimized hyperparameters
#### hyper parameter optimization - grid search
#### reflect the daily estimation and portfolio adjustment assumption
lambda_l_list = list(10.0**(-np.arange(1,2,1)))
lambda_b_list = list(10.0**(-np.arange(0,1,1)))
lambda_o_list = list(10.0**(-np.arange(0,1,1)))
TSLA_DP_DV_OH = OptHyper(TSLA_DP_DV,lambda_l_list = lambda_l_list,lambda_b_list = lambda_b_list,lambda_o_list = lambda_o_list,val_ratio=0.01)
TSLA_DP_DV_OH.opt_hyper_param

(0.1, 1.0, 1.0)

In [49]:
TSLA_DP_DV_OH_MF = ModelFit(TSLA_DP_DV,lambda_l = TSLA_DP_DV_OH.opt_hyper_param[0],lambda_b = TSLA_DP_DV_OH.opt_hyper_param[1], lambda_o=TSLA_DP_DV_OH.opt_hyper_param[2])
np.mean(TSLA_DP_DV_OH_MF.MSE_pred)

0.006123486094820377

In [51]:
TSLA_DP_DV_OH_MF.y_pred

[0.0077674403,
 0.012905525,
 0.011797203,
 -0.004692573,
 0.00570172,
 0.016765108,
 -0.0019329172,
 -0.0027861935,
 0.0008858223,
 0.0058305673]

In [53]:
TSLA_DP_DV_OH_MF.y_true

[0.06355291291751475,
 -0.04452763743232491,
 -0.04842373713092618,
 -0.04862218183112754,
 -0.03779928897736162,
 -0.05844972237400059,
 0.1964121085077153,
 -0.008195046455057975,
 0.04721129560609172,
 -0.00839050217557713]